In [1]:
%load_ext rpy2.ipython
import rpy2.robjects.lib.ggplot2 as ggplot2
import childespy
import numpy as np
import os
import imp
import pandas as pd
import Levenshtein
import itertools
import re

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 20614 bytes (20 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

# Extract IPA Annotated Words from Providence

In [2]:
pvd_idx = childespy.get_sql_query('select * from corpus where name = "Providence"',
                                 db_version = "2020.1").iloc[0]['id']

R[write to console]: Using supported database version: '2020.1'.



In [3]:
regenerate = True

if regenerate:
    pvd_chi_tokens = childespy.get_sql_query('select gloss, target_child_name, target_child_age, \
    speaker_code, actual_phonology, model_phonology, transcript_id, utterance_id, \
    token_order from token where speaker_code = "CHI" and corpus_id = '+str(pvd_idx),
        db_version = "2020.1")
    pvd_chi_tokens.to_csv('csv/pvd_tokens.csv', index=False)
else: 
    pvd_chi_tokens = pd.read_csv('csv/pvd_tokens.csv')

R[write to console]: Using supported database version: '2020.1'.



In [4]:
excludes = ['*','(.)','(..)', '(...)','(....)','(.....)'] # these are either unlabeled, or marked for number of syllables (but not for phonology)
pvd_chi_tokens = pvd_chi_tokens.loc[~(pvd_chi_tokens.model_phonology.isin(excludes) |
    pvd_chi_tokens.actual_phonology.isin(excludes))]

In [5]:
pvd_chi_tokens.iloc[0]

gloss                   Mommy
target_child_name        Alex
target_child_age        514.0
speaker_code              CHI
actual_phonology          ɑmɪ
model_phonology         mɑmiː
transcript_id           42204
utterance_id         16759315
token_order                 1
Name: 1, dtype: object

In [6]:
# get the most common transcription for each form
pvd_chi_tokens.gloss = [x.lower() for x in pvd_chi_tokens.gloss]
ipa_for_glosses = pvd_chi_tokens.groupby(['gloss']).model_phonology.agg(lambda x:
    x.value_counts().reset_index().iloc[0]['index']    
).reset_index()

In [7]:
print(ipa_for_glosses.shape)
test = ipa_for_glosses.dropna()
test.shape

(7929, 2)


(7929, 2)

In [8]:
test_words = ['dog','cat','box','fishes', 'mommy', 'potato','a', 'read']
ipa_for_glosses.loc[ipa_for_glosses.gloss.isin(test_words)]

,gloss,model_phonology
4,a,ə
768,box,bɑks
1072,cat,kæt
1892,dog,dɑɡ
2420,fishes,fɪʃəz
4340,mommy,mɑmiː
5271,potato,pəteɪtoʊ
5507,read,ɹiːd


In [9]:
ipa_for_glosses = ipa_for_glosses.loc[ipa_for_glosses.model_phonology != ""]
ipa_for_glosses['model_phonology'] = [x.replace("ː","").replace('ʌ','ə')
.replace('ɪ','ə').replace('ɔ','ɑ') for x in ipa_for_glosses['model_phonology']]
ipa_for_glosses.shape


(6746, 2)

In [10]:
ipa_for_glosses.iloc[0:3]

,gloss,model_phonology
4,a,ə
14,abe,eəb
15,abide,əbaəd


Using just the model phonology from Providence is insufficient. So we map to citation forms in the CMU pronouncing dictionary.

# Link to CMU Citation Forms

In [11]:
cmu = pd.read_csv('phon/cmu_dict_df.csv') # load an unedited version of the CMU pronunciation dictionary
cmu.columns = ['index','word','pronunciation']
from string import digits
remove_digits = str.maketrans('', '', digits)
cmu['phones'] = [[y.translate(remove_digits) for y in x.split(' ')] for x in cmu['pronunciation']] # remove indicators of stress marking 
cmu.word = [str(x).lower() for x in cmu.word]

In [12]:
# Output a CSV with all phonemes in the CMU (phon/phon_map.csv). Manually map these to phonemes and then load it (phon/phon_map_populated.csv)

# def flatten(list_of_lists):
#     return([item for subl in list_of_lists for item in subl])

# phone_inventory = pd.DataFrame({'arpa':np.unique(flatten(cmu.phones))})
# phone_inventory.to_csv('phon/phon_map.csv', index=False)


# Character differences / shorthand
# [X] cmu: ɝ -> əɹ
# [X] cmu:  tʃ -> ʧ
# [X] cmu  dʒ -> ʤ      
# Collapses
# [X] cmu  I -> schwa
# [X] ɑ in model phonlogy vs ɔ in cmu: convert ɔ to ɑ in both
# [X] model phonology: ʌ ->  ə

In [13]:
phone_map_df = pd.read_csv('phon/phon_map_populated.csv')
phone_map = dict(zip(phone_map_df.arpa, phone_map_df.ipa))
cv_map = dict(zip(phone_map_df.arpa, phone_map_df.c_or_v))
cmu['ipa'] = [[phone_map[x] for x in y] for y in cmu.phones]
cmu['structure'] = [[cv_map[x] for x in y] for y in cmu.phones]
cmu['num_vowels'] = [np.sum(np.array(x) == 'v') for x in cmu['structure']]
cmu['ipa_short'] = [''.join(x) for x in cmu['ipa']]
cmu['ipa_short'] =  [x.replace('ɝ', 'əɹ').replace('ɪ','ə').replace('ɔ','ɑ').replace('ər', 'əɹ').replace('er','əɹ') for x in cmu['ipa_short']]
# remove any parenthetical + contents from the word record in CMU
cmu['word'] = [re.sub("\(.+\)$", '', x) for x in cmu['word']]

In [14]:
# confirm that we have multiple pronunciation entries for a word like "read"
cmu.loc[cmu.word=="read"]

,index,word,pronunciation,phones,ipa,structure,num_vowels,ipa_short
97825,97826,read,R EH1 D,"[R, EH, D]","[ɹ, ɛ, d]","[c, v, c]",1,ɹɛd
97826,97827,read,R IY1 D,"[R, IY, D]","[ɹ, i, d]","[c, v, c]",1,ɹid


In [15]:
cmu.loc[cmu.word.isin(test_words)]

,index,word,pronunciation,phones,ipa,structure,num_vowels,ipa_short
70,71,a,AH0,[AH],[ə],[v],1,ə
71,72,a,EY1,[EY],[eɪ],[v],1,eə
14024,14025,box,B AA1 K S,"[B, AA, K, S]","[b, ɑ, k, s]","[c, v, c, c]",1,bɑks
19019,19020,cat,K AE1 T,"[K, AE, T]","[k, æ, t]","[c, v, c]",1,kæt
33320,33321,dog,D AO1 G,"[D, AO, G]","[d, ɔ, ɡ]","[c, v, c]",1,dɑɡ
42452,42453,fishes,F IH1 SH AH0 Z,"[F, IH, SH, AH, Z]","[f, ɪ, ʃ, ə, z]","[c, v, c, v, c]",2,fəʃəz
42453,42454,fishes,F IH1 SH IH0 Z,"[F, IH, SH, IH, Z]","[f, ɪ, ʃ, ɪ, z]","[c, v, c, v, c]",2,fəʃəz
79787,79788,mommy,M AA1 M IY0,"[M, AA, M, IY]","[m, ɑ, m, i]","[c, v, c, v]",2,mɑmi
93614,93615,potato,P AH0 T EY1 T OW2,"[P, AH, T, EY, T, OW]","[p, ə, t, eɪ, t, oʊ]","[c, v, c, v, c, v]",3,pəteətoʊ
97825,97826,read,R EH1 D,"[R, EH, D]","[ɹ, ɛ, d]","[c, v, c]",1,ɹɛd


## Compute Proportion of Providence words that are in the adjusted CMU 

In [16]:
found_words = ipa_for_glosses.merge(cmu, left_on=['gloss'],
    right_on = ['word'])
found_words.drop_duplicates(subset=['word'])
# drop duplicates for a given word

found_words.shape[0] / ipa_for_glosses.shape[0]

0.9540468425733768

Approximately 95% of the words in Providence are also present in CMU

In [17]:
# different pronunciations
missing_pronunctiations = ipa_for_glosses.loc[~ipa_for_glosses.gloss.isin(found_words.gloss)]
missing_pronunctiations_short = missing_pronunctiations[['gloss', 'model_phonology']]
missing_pronunctiations_short.sample(10)

,gloss,model_phonology
6517,stewie,stuwi
2965,gumga,ɡəmɡɑ
5437,rabbit's,ɹæbəts
2351,fewdinee,fjudini
3610,kinana,kinɑnə
6418,squiggly,skwəɡli
6698,synonyms,sənənəmz
908,bursie,bəɹsi
1849,dipsy,dəpsi
2312,fasih,fɑsi


These are justifiably odd words

## Compute Proportion of Providence annotations that match the annotation in adjusted CMU 

In [18]:
matched_words = ipa_for_glosses.merge(cmu, left_on=['gloss','model_phonology'],
    right_on = ['word','ipa_short'])
matched_words.shape[0] / ipa_for_glosses.shape[0]

0.8651052475541061

Approximately 86\% of words have the same pronunciation in the corrected CMU as in Providence

In [19]:
# different pronunciations
different_pronunctiations = ipa_for_glosses.loc[~ipa_for_glosses.gloss.isin(matched_words.gloss)]
different_pronunctiations = different_pronunctiations.merge(cmu, left_on=['gloss'], right_on=['word'])
different_pronunctiations_short = different_pronunctiations[['gloss', 'model_phonology','ipa_short']]
different_pronunctiations_short.columns = ['gloss','pvd_ipa','cmu_ipa']
different_pronunctiations_short.sample(10)

,gloss,pvd_ipa,cmu_ipa
132,this'll,ðəsɫ,ðəsəl
97,microwave,maəkɹəwweəv,maəkɹəweəv
55,gentlemen,ʤɛntəlmɛn,ʤɛnəlmən
57,genuine,ʤɛnjəwwən,ʤɛnjuaən
2,actually,ækʧəwəli,ækʧli
14,ba,bæ,bieə
53,garrett,ɡɛ,ɡɛɹət
24,camera's,kæməɹəz,kæmɹəz
67,hola,olɑ,hoʊlə
131,they'll,ðeəl,ðɛl


In [21]:
childes_counts = pd.read_csv('data/vocab.csv')
childes_counts.columns 
cmu_in_childes = cmu.loc[(cmu.word.isin(
    childes_counts.loc[childes_counts['count'] > 3].word)) & (cmu['num_vowels'] <=2)]

In [22]:
cmu_in_childes.shape

(13894, 8)

# Compare with Coverage in the BERT Tokenizer

In [24]:
from transformers import BertTokenizer
initial_tokenizer = BertTokenizer.from_pretrained('models/model_output2')

In [25]:
bert_tokenizer_vocab = initial_tokenizer.get_vocab().keys()

In [26]:
cmu_in_childes.shape

(13894, 8)

In [27]:
cmu_in_childes_with_tokenizer = cmu.loc[(cmu.word.isin(
    childes_counts.loc[childes_counts['count'] > 3].word)) & (cmu['num_vowels'] <=2) &
            (cmu.word.isin(bert_tokenizer_vocab))]
cmu_in_childes_with_tokenizer.shape

(8943, 8)

In [28]:
cmu_in_childes_with_tokenizer.to_csv('phon/cmu_in_childes.csv',index=False)

In [29]:
cmu_in_childes_with_tokenizer.loc[cmu_in_childes_with_tokenizer.word =='read']

,index,word,pronunciation,phones,ipa,structure,num_vowels,ipa_short
97825,97826,read,R EH1 D,"[R, EH, D]","[ɹ, ɛ, d]","[c, v, c]",1,ɹɛd
97826,97827,read,R IY1 D,"[R, IY, D]","[ɹ, i, d]","[c, v, c]",1,ɹid


# Compute Coverage of MacArthur CDI words

In [30]:
wswg  = pd.read_csv('data/WSWG_50percentproducing.csv') # load a list of all Words and Sentences + Words and Gestures words

In [31]:
wswg['word'] = [x.split(' ')[0].split('/')[0] for x in wswg.word]
' '.join(wswg['word'])
wswg_cmu = wswg.merge(cmu_in_childes)
wswg_cmu = wswg_cmu.drop_duplicates(subset=['word'])
print('Total coverage of MacArthur CDI words, before intersecting with BERT tokenizer')
print(wswg_cmu.shape[0] / wswg.shape[0])

Total coverage of MacArthur CDI words, before intersecting with BERT tokenizer
0.8918495297805643


In [32]:
wswg['word'] = [x.split(' ')[0].split('/')[0] for x in wswg.word]
' '.join(wswg['word'])
wswg_cmu = wswg.merge(cmu_in_childes_with_tokenizer)
wswg_cmu = wswg_cmu.drop_duplicates(subset=['word'])
print('Total coverage of MacArthur CDI words, after intersecting with BERT tokenizer')
print(wswg_cmu.shape[0] / wswg.shape[0])

Total coverage of MacArthur CDI words, after intersecting with BERT tokenizer
0.8197492163009404


In [33]:
# What words from the CDI are missing? 
' '.join(wswg.loc[~wswg.word.isin(wswg_cmu.word)].word)

"woof yum grrr moo meow banana vroom ouch quack baa shh cracker peekaboo owie choo diaper child's telephone potty pet's tummy toothbrush bicycle TV yucky cereal bathtub buttocks elephant butterfly crayon teddybear tickle pajamas cheerios yogurt bib carrots I raisin strawberry giraffe firetruck medicine pancake animal potato spaghetti hamburger helicopter babysitter's cockadoodledoo stroller potato motorcycle applesauce alligator pattycake popsicle crib refrigerator pretzel don't pickle muffin lemme donut slipper snowman radio vitamins sandbox lollipop underpants mittens fireman dryer jello melon another mailman sprinkler tricycle tomorrow mop hafta sneaker sled babysitter"

We are missing multisyllable words, onomatopoeia from CHILDES

# Test the Levenshtein Distance Function

In [35]:
def get_levdists(target, cmu_in_childes):
    print(target)
    distances = [Levenshtein.distance(target,x) for x in cmu_in_childes.ipa_short]
    dist_df = pd.DataFrame({'levdist':distances,'candidate':cmu_in_childes.word, 'ipa_short':cmu_in_childes.ipa_short})
    dist_df['target'] = target
    return(dist_df.sort_values(by=['levdist']))

goat_test = get_levdists('ɡoʊt', cmu_in_childes_with_tokenizer)
goat_test

ɡoʊt


,levdist,candidate,ipa_short,target
48256,0,goat,ɡoʊt,ɡoʊt
48241,1,go,ɡoʊ,ɡoʊt
48261,1,goats,ɡoʊts,ɡoʊt
79395,1,moat,moʊt,ɡoʊt
127639,1,vote,voʊt,ɡoʊt
...,...,...,...,...
39842,9,explained,əkspleənd,ɡoʊt
30688,9,described,dəskɹaəbd,ɡoʊt
67440,9,landscape,lændskeəp,ɡoʊt
115496,10,strangers,stɹeənʤəɹz,ɡoʊt
